<a href="https://colab.research.google.com/github/kardwalker/Lunar_model_wb/blob/main/Lunar_Model_WB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt install swig cmake
!pip install -r https://raw.githubusercontent.com/huggingface/deep-rl-class/main/notebooks/unit1/requirements-unit1.txt


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
cmake is already the newest version (3.22.1-1ubuntu1.22.04.2).
Suggested packages:
  swig-doc swig-examples swig4.0-examples swig4.0-doc
The following NEW packages will be installed:
  swig swig4.0
0 upgraded, 2 newly installed, 0 to remove and 49 not upgraded.
Need to get 1,116 kB of archives.
After this operation, 5,542 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 swig4.0 amd64 4.0.2-1ubuntu1 [1,110 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 swig all 4.0.2-1ubuntu1 [5,632 B]
Fetched 1,116 kB in 1s (746 kB/s)
Selecting previously unselected package swig4.0.
(Reading database ... 123620 files and directories currently installed.)
Preparing to unpack .../swig4.0_4.0.2-1ubuntu1_amd64.deb ...
Unpacking swig4.0 (4.0.2-1ubuntu1) ...
Selecting previously unselected package swig.
Preparing to unpack .../swig_4.0.2-1ubunt

In [ ]:
!sudo apt-get update
!apt install python3-opengl
!apt install ffmpeg
!apt install xvfb
!pip install pyvirtualdisplay


In [ ]:
import gymnasium as gym
from huggingface_sb3 import load_from_hub, package_to_hub
from huggingface_hub import notebook_login
from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.monitor import Monitor

In [ ]:
!pip install wandb -Uq
import wandb
wandb.login()


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/wandb/analytics/sentry.py:90: SentryHubDeprecationWarning: `sentry_sdk.Hub` is deprecated and will be removed in a future major release. Please consult our 1.x to 2.x migration guide for details on how to migrate `Hub` usage to the new API: https://docs.sentry.io/platforms/python/migration/1.x-to-2.x
  self.hub = sentry_sdk.Hub(client)
/usr/local/lib/python3.10/dist-packages/notebook/utils.py:280: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  return LooseVersion(v) >= LooseVersion(check)
wandb: Using wandb-core as the SDK backend. Please refer to htt

True

In [ ]:
sweep_config = {
    "method" : "random",}

metric = {
    "name" : "loss",
    "goal" : "minimize"
}
sweep_config["metric"] = metric

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
parameter_dict = {
    "n_steps" : {
        "values" : [128, 256, 512, 1024, 2048, 4096 ]
    },
    "n_epochs" : {
        "values" : [1, 3, 6, 10]
    },
    "ent_coef" : {
        "values" : [0.1 , 0.001 , 0.05, 0.005]
    },
    "gamma" : {
        "values" : [0.93, 0.94,0.95, 0.96, 0.97, 0.98, 0.99, 0.999]
    },
    "gae_lambda" : {
        "values" : [0.8, 0.9, 0.92, 0.95, 0.98, 0.99, 1.0]
    },
    "batch_size" : {
        "values" : [16,32, 64, 128, 256, 512, 1024, ]}



    }

In [ ]:
sweep_config['parameters'] = parameter_dict
sweep_id = wandb.sweep(sweep_config, project = "ppo-LunarLander-v2")


Create sweep with ID: b0yos0a9
Sweep URL: https://wandb.ai/kardwalk1979-IIT%20ISM%20/ppo-LunarLander-v2/sweeps/b0yos0a9


In [ ]:
import gymnasium as gym
env = gym.make("LunarLander-v2")

env.reset()






(array([ 0.00278673,  1.3982326 ,  0.2822405 , -0.56388736, -0.00322224,
        -0.06393168,  0.        ,  0.        ], dtype=float32),
 {})

In [ ]:
def train():
  wandb.init()
  env = make_vec_env("LunarLander-v2" ,n_envs = 16 )


  model = PPO(policy = "MlpPolicy",
            env = env,
            n_steps = wandb.config.n_steps,
            batch_size = wandb.config.batch_size,
            n_epochs = wandb.config.n_epochs,
            gamma =wandb.config.gamma,
            gae_lambda = wandb.config.gae_lambda,
            ent_coef = wandb.config.ent_coef,
            verbose = 1)

  model.learn(total_timesteps = 100000)
  model_name = "ppo-LunarLander-v2"
  model.save(model_name)
  wandb.save(model_name)


In [ ]:
wandb.agent(sweep_id, function = train)

wandb: Agent Starting Run: rutzmx9c with config:
wandb: 	batch_size: 16
wandb: 	ent_coef: 0.05
wandb: 	gae_lambda: 1
wandb: 	gamma: 0.93
wandb: 	n_epochs: 3
wandb: 	n_steps: 512


Using cuda device
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 91.9     |
|    ep_rew_mean     | -189     |
| time/              |          |
|    fps             | 2028     |
|    iterations      | 1        |
|    time_elapsed    | 4        |
|    total_timesteps | 8192     |
---------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 96.3         |
|    ep_rew_mean          | -174         |
| time/                   |              |
|    fps                  | 1195         |
|    iterations           | 2            |
|    time_elapsed         | 13           |
|    total_timesteps      | 16384        |
| train/                  |              |
|    approx_kl            | 0.0057441164 |
|    clip_fraction        | 0.0339       |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.38        |
|    explained_variance   | -0.00212    

wandb: Agent Starting Run: yvxu99tb with config:
wandb: 	batch_size: 128
wandb: 	ent_coef: 0.1
wandb: 	gae_lambda: 0.95
wandb: 	gamma: 0.95
wandb: 	n_epochs: 1
wandb: 	n_steps: 2048


Using cuda device
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 90.2     |
|    ep_rew_mean     | -199     |
| time/              |          |
|    fps             | 4194     |
|    iterations      | 1        |
|    time_elapsed    | 7        |
|    total_timesteps | 32768    |
---------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 96.2         |
|    ep_rew_mean          | -173         |
| time/                   |              |
|    fps                  | 3534         |
|    iterations           | 2            |
|    time_elapsed         | 18           |
|    total_timesteps      | 65536        |
| train/                  |              |
|    approx_kl            | 0.0015282531 |
|    clip_fraction        | 0.0022       |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.38        |
|    explained_variance   | 0.0126      

wandb: Agent Starting Run: sfxjxwp7 with config:
wandb: 	batch_size: 64
wandb: 	ent_coef: 0.1
wandb: 	gae_lambda: 0.8
wandb: 	gamma: 0.99
wandb: 	n_epochs: 1
wandb: 	n_steps: 128


...

Using cuda device
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 92.6     |
|    ep_rew_mean     | -194     |
| time/              |          |
|    fps             | 3328     |
|    iterations      | 1        |
|    time_elapsed    | 4        |
|    total_timesteps | 16384    |
---------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 98.1         |
|    ep_rew_mean          | -174         |
| time/                   |              |
|    fps                  | 2899         |
|    iterations           | 2            |
|    time_elapsed         | 11           |
|    total_timesteps      | 32768        |
| train/                  |              |
|    approx_kl            | 0.0061159083 |
|    clip_fraction        | 0.0224       |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.38        |
|    explained_variance   | -0.000384   

wandb: Agent Starting Run: 1wzp01s8 with config:
wandb: 	batch_size: 64
wandb: 	ent_coef: 0.1
wandb: 	gae_lambda: 0.99
wandb: 	gamma: 0.99
wandb: 	n_epochs: 3
wandb: 	n_steps: 128


Using cuda device
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 83       |
|    ep_rew_mean     | -204     |
| time/              |          |
|    fps             | 3000     |
|    iterations      | 1        |
|    time_elapsed    | 0        |
|    total_timesteps | 2048     |
---------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 83.7         |
|    ep_rew_mean          | -175         |
| time/                   |              |
|    fps                  | 2193         |
|    iterations           | 2            |
|    time_elapsed         | 1            |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0010684008 |
|    clip_fraction        | 0            |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.39        |
|    explained_variance   | -0.00117    

wandb: Agent Starting Run: nd9mrox2 with config:
wandb: 	batch_size: 128
wandb: 	ent_coef: 0.001
wandb: 	gae_lambda: 0.99
wandb: 	gamma: 0.93
wandb: 	n_epochs: 3
wandb: 	n_steps: 2048


Using cuda device
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 92.8     |
|    ep_rew_mean     | -180     |
| time/              |          |
|    fps             | 4027     |
|    iterations      | 1        |
|    time_elapsed    | 8        |
|    total_timesteps | 32768    |
---------------------------------


In [ ]:
print("fvjhcvghjc")
!pip install --upgrade ipython

In [ ]:
env = make_vec_env("LunarLander-v2" ,n_envs = 16 )
model = PPO(policy = "MlpPolicy",
            env = env,
            n_steps = 256,
            batch_size = 1024,
            n_epochs = 10,
            gamma =0.99,
            gae_lambda = 0.98,
            ent_coef = 0.1,
            verbose = 1)

In [ ]:
model.learn(total_timesteps = 100000)
model_name = "ppo-LunarLander-v2_1"
model.save(model_name)

In [ ]:
eval_env = gym.make("LunarLander-v2")
mean_reward , std_reward = evaluate_policy(model, eval_env, n_eval_episodes = 10, deterministic = True)
print(f"mean_reward {mean_reward} - std_reward {std_reward}")

In [ ]:
print("hfbdgjkbgjk")



/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
notebook_login()
!git config --global credential.helper store


In [ ]:
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.env_util import make_vec_env

from huggingface_sb3 import package_to_hub

repo_id =  "kardwalker/ppo-LunarLander-v2-1"
env_id = "LunarLander-v2"
eval_env = DummyVecEnv([lambda: gym.make(env_id, render_mode = "rgb_array")])
model_architecture = "PPO"
commit_message = "Upload PPO LunarLander-v2 trained agent"

package_to_hub(model = model,
               model_name = model_name,
               model_architecture = model_architecture,
               env_id = env_id,
               eval_env = eval_env,
               repo_id = repo_id,
               commit_message = commit_message)



/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


pytorch_variables.pth:   0%|          | 0.00/864 [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

policy.pth:   0%|          | 0.00/43.6k [00:00<?, ?B/s]

policy.optimizer.pth:   0%|          | 0.00/88.0k [00:00<?, ?B/s]

ppo-LunarLander-v2.zip:   0%|          | 0.00/148k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/kardwalker/ppo-LunarLander-v2/commit/b3d0554d0c9a9c556eea7134934109e26998ea52', commit_message='Upload PPO LunarLander-v2 trained agent', commit_description='', oid='b3d0554d0c9a9c556eea7134934109e26998ea52', pr_url=None, pr_revision=None, pr_num=None)